<a href="https://colab.research.google.com/github/nyp-sit/iti107-2024S2/blob/main/session-4/contextual_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contextual Embedding

One of the main drawbacks of embeddings such as Word2Vec and GloVE are that they have the same embedding for the same word regardless of its meaning in a particular context. For example, the word `rock` in `The rock concert is being held at national stadium` have a very different meaning in `The naughty boy throws a rock at the dog`.

Contextual embedding such as those produced by transformers (where the modern-day large language are based on) took into account the context of the word, and different embedding is generated for the same word depending on the context.

## Install Hugging Face Transformers library
If you are running this notebook in Google Colab, you will need to install the Hugging Face transformers library as it is not part of the standard environment.

In [ ]:
%%capture
!pip install transformers
!pip install datasets

Let's try to generate some embeddings using one of the transformer model `deberta`.

In [ ]:
from transformers import AutoModel, AutoTokenizer
# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Load a language model
model = AutoModel.from_pretrained("distilbert-base-uncased")
model = model.to('cuda')
# Tokenize the sentence
tokens = tokenizer('The rock concert is being held at national stadium.', return_tensors='pt')
print(tokens)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

{'input_ids': tensor([[ 101, 1996, 2600, 4164, 2003, 2108, 2218, 2012, 2120, 3346, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS]
the
rock
concert
is
being
held
at
national
stadium
.
[SEP]


We will pass the tokens through the model to generate embeddings.  We will take the embedding produced by the last layer.

In [ ]:
# Process the tokens
embeddings_1 = model(**tokens)[0]
print(embeddings_1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

**Questions**

1. What is the shape of the embeddings?
2. Why is the shape is such?

Let's try to find the embedding of the token 'rock' used here.

In [ ]:
embedding_rock1 = embeddings_1[0][2]
print(embedding_rock1)

Now write codes to find the embeddings of the word `rock` as used in the sentence `The naughty boy throws a rock at the dog.` and `The boy throws the rock into the drain`.


In [ ]:
tokens = tokenizer('The naughty boy throws a rock at the dog.', return_tensors='pt')
print(tokens)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))
tokens.to('cuda')
embeddings_2 = model(**tokens)[0]
embedding_rock2 = embeddings_2[0][6]
print(embedding_rock2)

In [ ]:
tokens = tokenizer('A big rock falls from the slope after heavy rain.', return_tensors='pt')
print(tokens)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))
embeddings_3 = model(**tokens)[0]
embedding_rock3 = embeddings_3[0][3]
print(embedding_rock3)

Let's compute how similar are the embeddings to each other

In [ ]:
import torch

cos = torch.nn.CosineSimilarity(dim=0)
similarity1 = cos(embedding_rock1, embedding_rock2)
print(similarity1)

similarity2 = cos(embedding_rock2, embedding_rock3)
print(similarity2)



We can see that embedding_rock2 are more similar to embedding_rock3 than with embedding_rock1.

In [ ]:
from datasets import load_dataset

# downloaded the datasets.
test_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_test.csv'
train_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_train.csv'

dataset = load_dataset('csv', data_files=train_data_url, split="train").shuffle().select(range(2000))

In [ ]:
dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 2000
})

In [ ]:
def process_dataset(sample):
    sample['sentiment'] = 0 if sample['sentiment'] == 'negative' else 1
    return sample

dataset = dataset.map(process_dataset)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train'].batch(batch_size=5)
test_dataset = dataset['test'].batch(batch_size=5)

Batching examples:   0%|          | 0/1600 [00:00<?, ? examples/s]

Batching examples:   0%|          | 0/400 [00:00<?, ? examples/s]

## Tokenization

We will now load the DistilBert tokenizer for the pretrained model "distillbert-base-uncased".  This is the same as the other lab exercise.

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased').to('cuda')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Here we will tokenize the text string, and pad the text string to the longest sequence in the batch, and also to truncate the sequence if it exceeds the maximum length allowed by the model (in BERT's case, it is 512).

In [ ]:
train_X = []
train_y = []

for data in train_dataset:
    train_encodings = tokenizer(data['review'], padding="max_length", max_length=512, truncation=True, return_tensors='pt').to('cuda')
    train_labels = data['sentiment']
    # train_encodings.to('cuda')
    # train_labels.to('cuda')
    outputs = model(**train_encodings)
    train_X.append(outputs[0].detach().cpu().numpy())
    train_y.append(train_labels)



In [ ]:
import torch

test_X = []
test_y = []

for data in test_dataset:
    test_encodings = tokenizer(data['review'], padding="max_length", max_length=512, truncation=True, return_tensors='pt').to('cuda')
    test_labels = data['sentiment']
    # train_encodings.to('cuda')
    # train_labels.to('cuda')
    outputs = model(**train_encodings)
    mean = torch.mean(outputs[0], axis=1)
    # test_X.append(outputs[0].detach().cpu().numpy())
    test_X.append(mean.detach().cpu().numpy())
    test_y.append(test_labels)



In [ ]:
# prompt: tf,reduce_mean equivalent for pytorch

import torch

# Assuming 'similarity1' and 'similarity2' are your tensors
similarity1 = torch.tensor(similarity1)
similarity2 = torch.tensor(similarity2)

# Calculate the mean of the similarities
mean_similarity = torch.mean(torch.stack([similarity1, similarity2]))

mean_similarity

In [ ]:
print(test_X.shape)
print(test_y.shape)

(400, 512, 768)
(400,)


## Train a classifier using the extracted features (embeddings)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(train_X, train_y)

ValueError: Found array with dim 3. LogisticRegression expected <= 2.

In [ ]:
print(f'train score : {clf.score(X_train, y_train)}')
print(f'validation score : {clf.score(X_val, y_val)}')
print(f'test score : {clf.score(X_test, y_test)}')

We should be getting an validation and accuracy score of around 86% to 87% which is quite good, considering we are training with only 2000 samples!

**Exercise**

1. Modify the code to use the hidden states from a different attention layer as features or take average of hidden states  from few layers as features.
2. Modify the code to use BERT model and see if it performs better than the DistilBERT. For BERT Model, the output of different layers are in `output[2]`